In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os  # For file handling
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# Download necessary NLTK data for sentiment analysis


In [ ]:
def load_stock_data(data_folder, stock_files):
    stock_data = {}  # Dictionary to store stock DataFrames
    
    for file in stock_files:
        stock_name = file.split("_")[0]  # Extract stock name (e.g., AAPL, AMZN)
        path = os.path.join(data_folder, file)
        
        # Load CSV into DataFrame
        df = pd.read_csv(path)
        
        # Drop 'Dividends' and 'Stock Splits' if they are all zeros
        if 'Dividends' in df.columns and df['Dividends'].nunique() == 1:
            df = df.drop(columns=['Dividends'])
        if 'Stock Splits' in df.columns and df['Stock Splits'].nunique() == 1:
            df = df.drop(columns=['Stock Splits'])
        
        # Convert 'Date' column to datetime.date format
        df['Date'] = pd.to_datetime(df['Date']).dt.date
        
        # Add cleaned DataFrame to dictionary
        stock_data[stock_name] = df
    
    return stock_data

# Define data folder and stock files
data_folder = "../../data/yfinance_data/yfinance_data"  # Adjust to your folder path
stock_files = [
    "AAPL_historical_data.csv", "AMZN_historical_data.csv", "GOOG_historical_data.csv",
    "META_historical_data.csv", "MSFT_historical_data.csv", "NVDA_historical_data.csv",
    "TSLA_historical_data.csv"
]

# Load stock data
stock_data = load_stock_data(data_folder, stock_files)


In [4]:
# Helper function to load news data
def load_news_data(news_file):
    # Load the CSV file
    news_df = pd.read_csv(news_file)

    # Display the first few rows of the date column to inspect the format (debugging)
    print(news_df['date'].head(10))

    # Convert the 'date' column to datetime, handling the timezone and retaining just the date
    news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce').dt.date

    # Drop rows where 'date' could not be parsed
    news_df = news_df.dropna(subset=['date'])

    # Retain relevant columns: 'date' and 'headline'
    news_df = news_df[['date', 'headline']]

    # Perform Sentiment Analysis
    news_df['sentiment_score'] = news_df['headline'].apply(lambda x: sia.polarity_scores(x)['compound'])

    # Group by date and calculate the average daily sentiment
    avg_sentiment = news_df.groupby('date')['sentiment_score'].mean().reset_index()
    avg_sentiment.rename(columns={'sentiment_score': 'avg_sentiment'}, inplace=True)

    return avg_sentiment


In [5]:
def analyze_correlation(stock_data, news_sentiment):
    results = {}
    for stock_name, stock_df in stock_data.items():
        # Merge stock data with news sentiment on dates
        merged_df = pd.merge(
            stock_df, news_sentiment, 
            left_on='Date', right_on='date', 
            how='inner'
        )
        
        # Drop redundant 'date' column
        merged_df = merged_df.drop(columns=['date'])
        
        # Calculate daily stock returns
        merged_df['Daily_Return'] = merged_df['Close'].pct_change()
        
        # Drop NaN values (due to percentage change)
        merged_df = merged_df.dropna()
        
        # Calculate Pearson correlation between sentiment and returns
        correlation = merged_df['Daily_Return'].corr(merged_df['avg_sentiment'])
        results[stock_name] = correlation
    
    return results

In [ ]:
# Define folder paths and file names
data_folder = "../../data/yfinance_data/yfinance_data"  # Adjust to your data folder path
news_file = os.path.join(data_folder, "raw_analyst_ratings.csv")

# List of stock files
stock_files = [
    "AAPL_historical_data.csv", "AMZN_historical_data.csv", 
    "GOOG_historical_data.csv", "META_historical_data.csv", 
    "MSFT_historical_data.csv", "NVDA_historical_data.csv", 
    "TSLA_historical_data.csv"
]

# Load stock data and news sentiment data
stock_data = load_stock_data(data_folder, stock_files)
news_sentiment = load_news_data(news_file)

# Perform correlation analysis
correlation_results = analyze_correlation(stock_data, news_sentiment)

# Display correlation results
print("\nCorrelation Between News Sentiment and Stock Returns:")
for stock, correlation in correlation_results.items():
    print(f"{stock}: {correlation:.4f}")


In [ ]:
# Visualize correlation results
plt.figure(figsize=(10, 6))
plt.bar(correlation_results.keys(), correlation_results.values(), color='skyblue')
plt.title("Correlation Between News Sentiment and Stock Returns")
plt.xlabel("Stock")
plt.ylabel("Correlation Coefficient")
plt.axhline(0, color='grey', linestyle='--')
plt.show()
